In [38]:
!pwd

/Volumes/MTS800/tmp/hust/study_at_hust/semesters/2025_1/DS/hello_ngoc_linh/vneconomy


In [39]:
!pip install beautifulsoup4 requests pandas numpy openpyxl openai python-dotenv selenium webdriver-manager httpx pydantic tqdm

In [40]:
# Import required libraries for web scraping and API calls
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
import json
import os
import re
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup
import time
import json

# Load environment variables from .env file
load_dotenv()

True

In [41]:
# Base URL for vneconomy website
BASE_URL = "https://vneconomy.vn/"

# Mapping of VN30 stock codes to their related keywords
KEYWORDS_MAP = {
    "ACB": ["ACB", "Ngân hàng ACB", "Ngân hàng TMCP Á Châu"],
    "BCM": ["BCM", "Becamex", "KCN Bình Dương", "khu công nghiệp Bình Dương", "VSIP", "Becamex IDC"],
    "BID": ["BIDV", "Ngân hàng Đầu tư và Phát triển Việt Nam"],
    "CTG": ["CTG", "VietinBank", "Ngân hàng Công Thương Việt Nam"],
    "DGC": ["DGC", "Hóa chất Đức Giang"],
    "FPT": ["FPT"],
    "GAS": ["PV GAS", "PV Gas", "Tổng Công ty Khí Việt Nam"],
    "GVR": ["GVR", "Tập đoàn Cao su", "Tập đoàn Công nghiệp Cao su Việt Nam"],
    "HDB": ["HDB", "HDBank", "Ngân hàng TMCP Phát triển Thành phố Hồ Chí Minh"],
    "HPG": ["HPG", "Hòa Phát"],
    "LPB": ["LPB", "LPBank", "LienVietPostBank", "Ngân hàng Bưu điện Liên Việt"],
    "MBB": ["MBB", "MBBank", "Ngân hàng Quân đội", "MB", "Ngân hàng TMCP Quân đội"],
    "MSN": ["MSN", "Masan", "WinCommerce"],
    "MWG": ["MWG", "Thế Giới Di Động", "Mobile World", "Bách Hóa Xanh", "BHX", "Điện Máy Xanh", "ĐMX", "TGDĐ"],
    "PLX": ["PLX", "Petrolimex", "Tập đoàn Xăng dầu Việt Nam"],
    "SAB": ["SAB", "Sabeco", "Tổng Công ty CP Bia - Rượu - Nước giải khát Sài Gòn"],
    "SHB": ["SHB", "Ngân hàng Thương mại Cổ phần Sài Gòn – Hà Nội", "Ngân hàng TMCP Sài Gòn Hà Nội"],
    "SSB": ["SSB", "Ngân hàng Thương mại Cổ phần Đông Nam Á", "Ngân hàng TMCP Đông Nam Á", "SeABank"],
    "SSI": ["SSI", "Chứng khoán SSI"],
    "STB": ["STB", "Sài Gòn Thương Tín", "Sacombank"],
    "TCB": ["TCB", "Techcombank", "Ngân hàng TMCP Kỹ Thương Việt Nam"],
    "TPB": ["TPB", "TPBank", "Ngân hàng Tiên Phong", "Ngân hàng TMCP Tiên Phong"],
    "VCB": ["VCB", "Vietcombank", "Ngân hàng TMCP Ngoại Thương Việt Nam", "Ngân hàng Ngoại thương"],
    "VHM": ["VHM", "Vinhomes"],
    "VIB": ["VIB", "Ngân hàng TMCP Quốc Tế Việt Nam", "Ngân hàng Quốc Tế"],
    "VIC": ["VIC", "Vingroup", "Công ty Cổ phần Tập đoàn Vingroup"],
    "VJC": ["VJC", "Vietjet Air", "Công ty Cổ phần Hàng không Vietjet", "máy bay Vietjet"],
    "VNM": ["VNM", "Vinamilk", "Công ty Cổ phần Sữa Việt Nam"],
    "VPB": ["VPB", "VPBank", "Ngân hàng TMCP Việt Nam Thịnh Vượng"],
    "VRE": ["VRE", "Vincom Retail", "Công ty Cổ phần Vincom Retail"]
}

# Limit category option
LIMIT_CATEGORY = True
NUM_CATEGORY = 5
MAX_PAGES_PER_CATEGORY = 5

# Date filtering option
FILTER_BY_DATE = False  # Set to False to disable date filtering

# Date range filter (inclusive)
START_DATE = datetime(2020, 1, 1)
END_DATE = datetime(2024, 12, 31)

In [42]:
def get_categories():
    # Fetch the homepage HTML
    html = requests.get(BASE_URL, timeout=10).text
    soup = BeautifulSoup(html, "html.parser")

    categories = []

    # Find all anchor tags with title attribute
    for a in soup.find_all("a", title=True):
        href = a.get("href", "")
        
        # Only keep links ending with .htm
        if href.endswith(".htm"):
            title = a.get("title").strip()

            # Normalize the URL to absolute path
            if href.startswith("/"):
                full_url = BASE_URL.rstrip("/") + href
            else:
                full_url = href

            categories.append({
                "title": title,
                "url": full_url
            })

    return categories


def filter_category_pages(categories):
    # Initialize OpenAI client with API key from environment variable
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    # Format categories as text for GPT
    urls_text = "\n".join([f"- {c['title']}: {c['url']}" for c in categories])

    # Call GPT to filter out article pages and keep only category pages
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {
                "role": "system",
                "content": "You are an assistant that filters a list of URLs. "
                           "Keep only main category pages and remove specific news articles. "
                           "Return only a JSON array of objects with 'title' and 'url'."
            },
            {
                "role": "user",
                "content": f"Here is the list:\n{urls_text}\n\nFilter them and return JSON array."
            }
        ],
        temperature=0
    )

    # Parse the JSON response
    filtered_json = response.choices[0].message.content
    try:
        filtered_list = json.loads(filtered_json)
    except json.JSONDecodeError:
        print("Failed to parse JSON. Here's raw output:")
        print(filtered_json)
        filtered_list = []

    return filtered_list


# Get all categories from the homepage
cats = get_categories()
# Filter to keep only main category pages
filtered_cats = filter_category_pages(cats)

# Display filtered categories
print("Filtered Categories:")
for c in filtered_cats:
    print(f"- {c['title']}: {c['url']}")

Filtered Categories:
- Chứng khoán: https://vneconomy.vn/chung-khoan.htm
- Tiêu & Dùng: https://vneconomy.vn/tieu-dung.htm
- VnE TV: https://vneconomy.vn/video.htm
- eMagazine: https://vneconomy.vn/emagazine.htm
- Infographics: https://vneconomy.vn/infographics.htm
- Kinh tế xanh: https://vneconomy.vn/kinh-te-xanh.htm
- Chuyển động xanh: https://vneconomy.vn/chuyen-dong-xanh.htm
- Pháp lý: https://vneconomy.vn/phap-ly-kinh-te-xanh.htm
- Thương hiệu xanh: https://vneconomy.vn/thuong-hieu-xanh.htm
- Diễn đàn: https://vneconomy.vn/dien-dan-kinh-te-xanh.htm
- Tiêu điểm: https://vneconomy.vn/tieu-diem.htm
- Tài chính: https://vneconomy.vn/tai-chinh.htm
- Ngân hàng: https://vneconomy.vn/tai-chinh-ngan-hang.htm
- Thị trường vốn: https://vneconomy.vn/thi-truong-von-tai-chinh.htm
- Thuế: https://vneconomy.vn/thue-tai-chhinh.htm
- Bảo hiểm: https://vneconomy.vn/bao-hiem-tai-chinh.htm
- Doanh nghiệp niêm yết: https://vneconomy.vn/doanh-nghiep-niem-yet.htm
- Thị trường: https://vneconomy.vn/thi-tr

In [43]:
def fetch_article_summary(article_url):
    # Fetch the meta description from article page as summary
    try:
        html = requests.get(article_url, timeout=10).text
        soup = BeautifulSoup(html, "html.parser")
        meta_desc = soup.find("meta", attrs={"name": "description"})
        if meta_desc and meta_desc.get("content"):
            return meta_desc["content"].strip()
    except Exception as e:
        print(f"Failed to fetch summary for {article_url}: {e}")
    return ""


def crawl_articles(category_url, max_pages=MAX_PAGES_PER_CATEGORY, delay=1):
    # Crawl all articles from a category with pagination
    articles = []

    for page in range(1, max_pages + 1):
        # Build pagination URL
        if "?" in category_url:
            url = f"{category_url}&page={page}"
        else:
            url = f"{category_url}?page={page}"

        print(f"Crawling {url} ...")
        try:
            html = requests.get(url, timeout=10).text
        except Exception as e:
            print(f"Failed to fetch {url}: {e}")
            break

        soup = BeautifulSoup(html, "html.parser")
        # Find all article links with responsive-image-link class
        article_tags = soup.find_all("a", class_="responsive-image-link", title=True, href=True)

        # Stop if no more articles found
        if not article_tags:
            print(f"  ⚠ No articles found on page {page}. Reached end of available pages.")
            break

        for a in article_tags:
            title = a.get("title").strip()
            href = a.get("href").strip()
            full_url = href if href.startswith("http") else f"https://vneconomy.vn{href}"

            # Fetch article summary
            summary = fetch_article_summary(full_url)

            articles.append({
                "title": title,
                "url": full_url,
                "summary": summary
            })

        # Delay between requests to avoid overwhelming the server
        time.sleep(delay)

    return articles


# Crawl articles from each category
all_articles = {}
cnt = 0
for cat in filtered_cats:
    print(f"Processing category: {cat['title']}")
    cat_articles = crawl_articles(cat['url'])
    all_articles[cat['title']] = cat_articles
    cnt += 1
    if LIMIT_CATEGORY:
        # Limit to specified number of categories
        if cnt >= NUM_CATEGORY:
            break

print("Done. Total categories:", len(all_articles))

Processing category: Chứng khoán
Crawling https://vneconomy.vn/chung-khoan.htm?page=1 ...
Crawling https://vneconomy.vn/chung-khoan.htm?page=2 ...
Crawling https://vneconomy.vn/chung-khoan.htm?page=2 ...
Crawling https://vneconomy.vn/chung-khoan.htm?page=3 ...
Crawling https://vneconomy.vn/chung-khoan.htm?page=3 ...
Crawling https://vneconomy.vn/chung-khoan.htm?page=4 ...
Crawling https://vneconomy.vn/chung-khoan.htm?page=4 ...
Crawling https://vneconomy.vn/chung-khoan.htm?page=5 ...
Crawling https://vneconomy.vn/chung-khoan.htm?page=5 ...
Processing category: Tiêu & Dùng
Crawling https://vneconomy.vn/tieu-dung.htm?page=1 ...
Processing category: Tiêu & Dùng
Crawling https://vneconomy.vn/tieu-dung.htm?page=1 ...
Crawling https://vneconomy.vn/tieu-dung.htm?page=2 ...
Crawling https://vneconomy.vn/tieu-dung.htm?page=2 ...
Crawling https://vneconomy.vn/tieu-dung.htm?page=3 ...
Crawling https://vneconomy.vn/tieu-dung.htm?page=3 ...
Crawling https://vneconomy.vn/tieu-dung.htm?page=4 ...
Cra

In [44]:
def extract_publish_date(article_url):
    """
    Extract publication date from article page's meta tag.
    Returns date in format YYYY-MM-DD or None if not found.
    """
    try:
        # Use proper browser headers to avoid 403 errors
        date_headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate, br',
            'Referer': 'https://vneconomy.vn/',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
        }
        
        response = requests.get(article_url, headers=date_headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find meta tag with article:published_time
        meta_tag = soup.find('meta', property='article:published_time')
        if meta_tag and meta_tag.get('content'):
            # Extract date part (YYYY-MM-DD) from datetime string
            datetime_str = meta_tag['content']
            date_part = datetime_str.split('T')[0]
            return date_part
        
        return None
    except Exception as e:
        print(f"  ✗ Error extracting date: {e}")
        return None


def is_date_in_range(date_str, start_date=START_DATE, end_date=END_DATE):
    """
    Check if date string is within the specified range.
    """
    if not date_str or date_str == 'N/A':
        return False
    try:
        article_date = datetime.strptime(date_str, '%Y-%m-%d')
        return start_date <= article_date <= end_date
    except:
        return False


def filter_and_group_by_codes(articles, keywords_map=KEYWORDS_MAP):
    """
    Filter articles by VN30 keywords and group by stock codes.
    Uses word boundaries to match full keywords only.
    Returns a list of dictionaries with stock code and associated articles.
    """
    # Dictionary to store articles for each code (using link as key to avoid duplicates)
    code_articles = {code: {} for code in keywords_map.keys()}
    
    for art in articles:
        # Combine title and summary for keyword matching
        text = (art["title"] + " " + art.get("summary", "")).lower()
        matched_codes = []
        
        # Check each stock code's keywords
        for code, kws in keywords_map.items():
            for kw in kws:
                # Use word boundary regex to match full words only
                pattern = r'\b' + re.escape(kw.lower()) + r'\b'
                if re.search(pattern, text):
                    matched_codes.append(code)
                    break
        
        # Add article to each matched code's dictionary (using link as key to prevent duplicates)
        article_link = art["url"]
        for code in matched_codes:
            if article_link and article_link not in code_articles[code]:
                code_articles[code][article_link] = {
                    'Stock_Code': code,
                    'Title': art["title"],
                    'Link': article_link,
                    'Summary': art.get("summary", "N/A")
                }
    
    # Flatten into single list, sorted by stock code
    result = []
    for code in sorted(code_articles.keys()):
        result.extend(code_articles[code].values())
    
    return result, {code: list(articles.values()) for code, articles in code_articles.items()}

In [45]:
# Flatten all articles from all categories into a single list
all_articles_list = []
for cat_articles in all_articles.values():
    all_articles_list.extend(cat_articles)

print(f"\n{'='*60}")
print("Filtering and grouping articles by VN30 stock codes...")
print(f"{'='*60}\n")

print(f"Total articles before filtering: {len(all_articles_list)}")

# Filter and group by codes
grouped_articles, code_dict = filter_and_group_by_codes(all_articles_list, keywords_map=KEYWORDS_MAP)

print(f"Articles matching VN30 keywords: {len(grouped_articles)}")

# Extract publication dates for filtered articles
if grouped_articles:
    print(f"\n{'='*60}")
    print("Extracting publication dates from article pages...")
    print(f"{'='*60}\n")
    
    for idx, article in enumerate(grouped_articles, 1):
        print(f"[{idx}/{len(grouped_articles)}] Extracting date for: {article['Title'][:50]}...")
        pub_date = extract_publish_date(article['Link'])
        article['Date'] = pub_date if pub_date else 'N/A'
        time.sleep(0.5)  # Delay between requests
    
    print("\n✓ Date extraction completed")
    
    # Filter by date range
    if FILTER_BY_DATE:
        print(f"\n{'='*60}")
        print(f"Filtering articles by date range: {START_DATE.strftime('%Y-%m-%d')} to {END_DATE.strftime('%Y-%m-%d')}")
        print(f"{'='*60}\n")
        
        articles_before_date_filter = len(grouped_articles)
        grouped_articles = [art for art in grouped_articles if is_date_in_range(art['Date'])]
        
        print(f"Articles before date filter: {articles_before_date_filter}")
        print(f"Articles after date filter: {len(grouped_articles)}")
    else:
        print(f"\n⚠ Date filtering is disabled (FILTER_BY_DATE=False)")

# Save to single CSV file
output_filename = "titles_vn30_vneconomy.csv"
if grouped_articles:
    df_output = pd.DataFrame(grouped_articles)
    # Reorder columns to have Date after Stock_Code
    cols = ['Stock_Code', 'Date', 'Title', 'Link', 'Summary']
    df_output = df_output[cols]
    df_output.to_csv(output_filename, index=False, encoding='utf-8-sig')
    print(f"✓ Saved to '{output_filename}'")
    
    # Recalculate statistics from date-filtered articles
    code_stats_filtered = {}
    for article in grouped_articles:
        code = article['Stock_Code']
        code_stats_filtered[code] = code_stats_filtered.get(code, 0) + 1
    
    # Print statistics by stock code
    print(f"\n{'='*60}")
    print("Statistics by stock code (after date filtering):")
    print(f"{'='*60}")
    for code in sorted(code_stats_filtered.keys()):
        count = code_stats_filtered[code]
        if count > 0:
            print(f"{code}: {count} articles")
    
    # Display sample
    print(f"\n{'='*60}")
    print("Sample articles (first 5):")
    print(f"{'='*60}")
    for idx, row in df_output.head(5).iterrows():
        print(f"\n[{row['Stock_Code']}] {row['Date']} - {row['Title']}")
        print(f"Link: {row['Link']}")
else:
    print("No articles matched VN30 keywords or date range!")


Filtering and grouping articles by VN30 stock codes...

Total articles before filtering: 835
Articles matching VN30 keywords: 7

Extracting publication dates from article pages...

[1/7] Extracting date for: NIM toàn ngành ngân hàng về mức thấp nhất kể từ cu...
Articles matching VN30 keywords: 7

Extracting publication dates from article pages...

[1/7] Extracting date for: NIM toàn ngành ngân hàng về mức thấp nhất kể từ cu...
[2/7] Extracting date for: Hơn một thập kỷ đồng hành cùng doanh nghiệp xuất n...
[2/7] Extracting date for: Hơn một thập kỷ đồng hành cùng doanh nghiệp xuất n...
[3/7] Extracting date for: Masan Consumer duy trì lợi nhuận bền vững giữa biế...
[3/7] Extracting date for: Masan Consumer duy trì lợi nhuận bền vững giữa biế...
[4/7] Extracting date for: SHB được dự báo vào rổ chỉ số toàn cầu FTSE Global...
[4/7] Extracting date for: SHB được dự báo vào rổ chỉ số toàn cầu FTSE Global...
[5/7] Extracting date for: Cổ phiếu blue-chips dẫn sóng, VN-Index đột phá mốc...
[